# Cached properties
The `@property` decorator is a very nice tool to create functions that look like attributes that are computed based on other attributes. However, sometimes, this computation can be quite heavy and slow down the program, especially if you have a property that depends on a property that depends on a property etc.
That's where the `@cached_property` from `functools` comes in. This stores the property as an attribute and doesn't recompute it every time it is accessed. However of course the downside is that sometimes we want it to be recomputed, for example when the value of the attribute on which it depends, changes. That problem can be solved by making a getter/setter for the attribute on which it depends. See the examples below.

In [1]:
from functools import cached_property
from time import sleep

## Regular @property is recomputed every time

In [9]:
class Person():
    def __init__(self, name='Pol'):
        self.name = name
        
    @property
    def sons_name(self):
        sleep(4) # Make the program wait for 4 seconds to prove a point
        return self.name + 'son'

In [3]:
p1 = Person()

In [4]:
print("takes 4 seconds to return:")
p1.sons_name

takes 4 seconds to return:


'Polson'

In [5]:
print("and AGAIN takes 4 seconds to return:")
p1.sons_name

and AGAIN takes 4 seconds to return:


'Polson'

## @cached_property doesn't recompute, but also doesn't update when the value on which it depends, changes:

In [13]:
class Person():
    def __init__(self, name='Pol'):
        self.name = name
        
    @cached_property
    def sons_name(self):
        sleep(4)
        return self.name + 'son'

In [14]:
p1 = Person()

In [15]:
print("takes 4 seconds to return:")
p1.sons_name

takes 4 seconds to return:


'Polson'

In [16]:
print("Returns faster")
p1.sons_name

Returns faster


'Polson'

In [17]:
p1.name = 'Johan'

In [18]:
print("Son's name is not updated!")
p1.sons_name

Son's name is not updated!


'Polson'

## Solve this by deleting the @cached_property attribute in the setter of name:

In [19]:
class Person2():
    def __init__(self, name='pol'):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, x):
        self._name = x
        print("delete sons_name")
        if 'sons_name' in  self.__dict__:
            del self.sons_name
        
    @cached_property
    def sons_name(self):
        sleep(4)
        return self.name + 'son'

In [20]:
p2 = Person2()

In [21]:
print("takes 4 seconds to return:")
p2.sons_name

takes 4 seconds to return:


'polson'

In [22]:
print("Returns faster")
p2.sons_name

Returns faster


'polson'

In [23]:
p2.name = 'Marie'

delete sons_name


In [24]:
print("The son's name is updated!")
p2.sons_name

The son's name is updated!


'Marieson'

## what happens when we have a lot of properties?

In [25]:
class Person2():
    def __init__(self, name='pol'):
        self._name = 'pol'
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, x):
        self._name = x
        list_of_dependent_properties = ['sons_name',
                                        'daughters_name']
        for prop in list_of_dependent_properties:
            if prop in  self.__dict__:
                delattr(self,prop)
        
    @cached_property
    def sons_name(self):
        sleep(2)
        return self.name + 'son'
    
    @cached_property
    def daughters_name(self):
        sleep(4)
        return self.name + 'dottir'

In [26]:
p2 = Person2()

In [27]:
p2.sons_name

'polson'

In [28]:
p2.daughters_name

'poldottir'

In [29]:
p2.name = 'Claire'

In [30]:
p2.sons_name

'Claireson'

In [31]:
p2.daughters_name

'Clairedottir'

## Closing notes
I've shown an example where multiple `@cached_property` (`sons_name, daughters_name`) depend on the same attribute (`name`). So we just need to edit the setter of name when we add more `@cached_property`. But if a `@cached_property` depends on more than 1 attribute, they all need to have setters. This might get complicated...